## inserção dos dados CSV no Banco de dados

Importando as bibliotecas

In [1]:
import pandas as pd
import pyodbc

Lendo o CSV

In [ ]:
df = pd.read_csv(r"C:\Users\eunat\Desktop\Bank_Churn_Messy@.csv", delimiter=";")
df

Função para tratar coluna float desejada

In [ ]:
def limpar_coluna_float(df, coluna, casas_decimais=2):

    # 1. Substituir NaN por zero
    df[coluna] = df[coluna].fillna(0)
    
    # 2. Substituir Inf ou -Inf por zero
    df[coluna] = df[coluna].replace([np.inf, -np.inf], 0)
    
    # 3. Arredondar para o número especificado de casas decimais
    df[coluna] = df[coluna].round(casas_decimais)
    
    # 4. Verificar se a coluna tem valores não numéricos e removê-los (se necessário)
    df[coluna] = pd.to_numeric(df[coluna], errors='coerce') 
    
    # Opcional: Substituir novamente os NaN gerados após a conversão
    df[coluna] = df[coluna].fillna(0)

    return df

# Aplicar a limpeza na coluna
df = limpar_coluna_float(df, 'Balance', casas_decimais=2)

# Verificar o resultado
print(df['Balance'])


Verificar a tipagem das colunas

In [ ]:
for coluna, tipo in df.dtypes.items():
    print(f"{coluna}, Tipo: {tipo}")

Excluir coluna duplicada

In [23]:
df = df.drop(columns=["Tenure.1"])

Conexão com o banco de dados

In [ ]:
server = 'DESKTOP-QL6I1JD' 
database = 'Bank_Churn_Quality'  
conexaoDB = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};'
                      f'SERVER={server};'
                      f'DATABASE={database};'
                      'Trusted_Connection=yes;')

cursor = conexaoDB.cursor()  

Substituição de valores nas colunas

In [25]:
df["HasCrCard"] = df["HasCrCard"].map({"Yes": 1, "No": 0})
df["IsActiveMember"] = df["IsActiveMember"].map({"Yes": 1, "No": 0})

Deleta duplicadas

In [29]:
df = df.drop_duplicates()

"FOR" para inserir todos os dados do CSV no BD

In [ ]:
for index, linha in df.iterrows():
    try:
        cursor.execute(
            """
            INSERT INTO [Customers] (CustomerId, Surname, CreditScore, Geography, Gender, Age, Tenure, 
                                   EstimatedSalary, Balance, NumOfProducts, HasCrCard, IsActiveMember, Exited)
            VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
            """,
            linha.CustomerId, linha.Surname, linha.CreditScore, linha.Geography, linha.Gender, linha.Age,
            linha.Tenure, linha.EstimatedSalary, linha.Balance, linha.NumOfProducts,
            linha.HasCrCard, linha.IsActiveMember, linha.Exited
        )
    except Exception as e:
        print(f"Erro ao inserir o índice {index}: {e}")
        continue

cursor.commit()
cursor.close()
conexaoDB.close()
